d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Lab 3 - Sharing Insights
## Module 6 Assignment

In this lab, we will explore a small mock data set from a group of data centers. You'll see that is is similar to the data you have been working with, but it contains a few new columns and it is structured slightly differently to test your skills with hierarchical data manipulation. 

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this assignment you will: </br>

* Apply higher-order functions to array data
* Apply advanced aggregation and summary techniques to process data
* Present data in an interactive dashboard or static file 

As you work through the following tasks, you will be prompted to enter selected answers in Coursera. Find the quiz associated with this lab to enter your answers. 

Run the cell below to prepare this workspace for the lab.

In [0]:
%run ../Includes/Classroom-Setup

### Exercise 1: Create a table

**Summary:** Create a table. 

Use this path to access the data: `/mnt/training/iot-devices/data-centers/energy.json`

Steps to complete: 
* Write a `CREATE TABLE` statement for the data located at the endpoint listed above
* Use json as the file format

In [0]:
%sql
--TODO
CREATE TABLE IF NOT EXISTS Energy
USING json                          
OPTIONS (
    path "/mnt/training/iot-devices/data-centers/energy.json",
    inferSchema "true"
    );

### Exercise 2: Sample the table

**Summary:** Sample the table to get a closer look at a few rows

Steps to complete: 
* Write a query that allows you to see a few rows of the data

In [0]:
%sql
--TODO
SELECT 
  *
FROM Energy
TABLESAMPLE (3 ROWS)

battery_level,co2_level,device_id,device_type,signal,temps,timestamp
"List(3, 3, 2)","List(1343, 1595, 1405)",0,sensor-istick,"List(24, 24, 25)","List(22, 23, 21, 23)",2019/08/02 15:00:00
"List(1, 1, 2)","List(1213, 1346, 1247)",1,sensor-inest,"List(22, 24, 24)","List(22, 37, 34, 39)",2019/07/01 03:00:00
"List(3, 1, 3)","List(1261, 1216, 1258)",2,sensor-ipad,"List(25, 24, 28)","List(32, 33, 40, 44)",2019/06/03 11:00:00


### Exercise 3: Create view

**Summary:** Create a temporary view that displays the timestamp column as a timestamp. 

Steps to complete: 
* Create a temporary view named `DCDevices`
* Convert the `timestamp` column to a timestamp type. Refer to the [Datetime patterns](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html#) documentation for the formatting information. 
* (Optional) Rename columns to use camelCase

In [0]:
%sql
--TODO

create or replace temporary view DCDevices
as
  SELECT 
  battery_level as batteryLevel,
  co2_level as co2Level,
  device_id as deviceId,
  device_type as deviceType,
  signal,
  temps,
  to_timestamp(timestamp, 'yyyy/MM/dd HH:mm:ss') as time
    --date_format(to_timestamp(timestamp,'yyyy/MM/dd HH:mm:ss'),"yyyy-MM-dd HH:mm:ss") as timeStamp
  FROM Energy;

select * from DCDevices limit 3

batteryLevel,co2Level,deviceId,deviceType,signal,temps,time
"List(3, 3, 2)","List(1343, 1595, 1405)",0,sensor-istick,"List(24, 24, 25)","List(22, 23, 21, 23)",2019-08-02T15:00:00.000+0000
"List(1, 1, 2)","List(1213, 1346, 1247)",1,sensor-inest,"List(22, 24, 24)","List(22, 37, 34, 39)",2019-07-01T03:00:00.000+0000
"List(3, 1, 3)","List(1261, 1216, 1258)",2,sensor-ipad,"List(25, 24, 28)","List(32, 33, 40, 44)",2019-06-03T11:00:00.000+0000


### Exercise 4: Flag records with defective batteries

**Summary:** When a battery is malfunctioning, it can report negative battery levels. Create a new boolean column `needService` that shows whether a device needs service.  

Steps to complete: 
* Write a query that shows which devices have malfunctioning batteries
* Include columns `batteryLevel`, `deviceId`, and `needService`
* Order the results by `deviceId`, and then `batteryLevel`
* **Answer the corresponding question in Coursera**

In [0]:
%sql
--TODO
SELECT
  batteryLevel,
  deviceId,
  EXISTS (batteryLevel, c -> c < 0) needService
FROM DCDevices
order by deviceId, batteryLevel
limit 10

batteryLevel,deviceId,needService
"List(-4, -1, -1)",0,true
"List(-4, -1, -1)",0,true
"List(-4, -1, 0)",0,true
"List(-3, -3, -4)",0,true
"List(-3, -2, -2)",0,true
"List(-3, -2, -2)",0,true
"List(-3, -2, -2)",0,true
"List(-3, -2, -1)",0,true
"List(-3, -1, -3)",0,true
"List(-3, -1, -1)",0,true


-sandbox
### Exercise 5: Display high CO<sub>2</sub> levels

**Summary:** Create a new column to display only CO<sub>2</sub> levels that exceed 1400 ppm. 

Steps to complete: 
* Include columns `deviceId`, `deviceType`, `highCO2`, `time`
* The column `highCO2` should contain an array of CO<sub>2</sub> readings over 1400
* Show only records that contain `highCO2` values
* Order by `deviceId`, and then `highCO2`

**Answer the corresponding question in Coursera**

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You may need to use a subquery to write this in a single query statement.

In [0]:
%sql
--TODO
select 
  deviceId, 
  deviceType,
  FILTER(co2Level, t -> t > 1400) highCO2,
  time
from
  DCDevices
where EXISTS(co2Level, t -> t > 1400) is not false
order by deviceId, highCO2
limit 3;

deviceId,deviceType,highCO2,time
0,sensor-ipad,List(1401),2019-06-29T10:00:00.000+0000
0,sensor-ipad,List(1401),2019-06-01T22:00:00.000+0000
0,sensor-istick,List(1401),2019-08-16T21:00:00.000+0000


### Exercise 6: Create a partitioned table

**Summary:** Create a new table partitioned by `deviceId`

Steps to complete: 
* Include all columns
* Create the table using Parquet
* Rename the partitioned column `p_deviceId`
* Run a `SELECT *`  to view your table. 

**Answer the corresponding question in Coursera**

In [0]:
%sql
--TODO
CREATE TABLE IF NOT EXISTS PartitionedTable
USING PARQUET
PARTITIONED BY (p_deviceId)
AS
  SELECT
    batteryLevel,
    co2Level,
    deviceId as p_deviceId,
    deviceType,
    signal,
    TRANSFORM (temps, t -> CAST(t AS int)) temps,
    time
  FROM DCDevices;
  
SELECT * FROM PartitionedTable limit 3;

batteryLevel,co2Level,deviceType,signal,temps,time,p_deviceId
"List(1, 1, 1)","List(1446, 1421, 1420)",sensor-igauge,"List(14, 14, 14)","List(33, 26, 27, 25)",2019-07-29T18:00:00.000+0000,0
"List(6, 7, 6)","List(1173, 1202, 1402)",sensor-istick,"List(27, 26, 26)","List(28, 30, 35, 34)",2019-06-10T03:00:00.000+0000,0
"List(2, 0, 2)","List(1189, 1121, 1071)",sensor-igauge,"List(15, 14, 14)","List(20, 24, 25, 22)",2019-07-30T02:00:00.000+0000,0


### Exercise 7: Visualize average temperatures

In [0]:
%sql
--TODO
create or replace temporary view VizAvg
as 
select
  p_deviceId,
  temps,
  REDUCE(temps, 0, (t, acc) -> t + acc, acc ->(acc div size(temps))) as avg_daily_temp_c
from PartitionedTable;

SELECT 
  p_deviceId,
  temps,
  avg_daily_temp_c,
  AVG(avg_daily_temp_c)
  OVER (PARTITION BY p_deviceId) AS avg_device_temp_c
FROM VizAvg

p_deviceId,temps,avg_daily_temp_c,avg_device_temp_c
0,"List(33, 26, 27, 25)",27,24.510245901639344
0,"List(28, 30, 35, 34)",31,24.510245901639344
0,"List(20, 24, 25, 22)",22,24.510245901639344
0,"List(32, 25, 28, 25)",27,24.510245901639344
0,"List(40, 43, 37, 41)",40,24.510245901639344
0,"List(24, 25, 30, 33)",28,24.510245901639344
0,"List(25, 25, 25, 21)",24,24.510245901639344
0,"List(24, 27, 25, 28)",26,24.510245901639344
0,"List(20, 16, 11, 33)",20,24.510245901639344
0,"List(26, 20, 25, 35)",26,24.510245901639344


Output can only be rendered in Databricks

### Exercise 8: Create a widget

In [0]:
%sql
--TODO
CREATE WIDGET DROPDOWN selectedDeviceId DEFAULT "0" CHOICES
SELECT
  DISTINCT p_deviceId
FROM
  VizAvg
order by p_deviceId

### Exercise 9: Use the widget in a query

In [0]:
%sql
--TODO
SELECT 
  p_deviceId,
  ROUND(AVG(avg_daily_temp_c),4) AS avgTemp,
  ROUND(STD(avg_daily_temp_c), 2) AS stdTemp
FROM VizAvg
WHERE p_deviceId = getArgument("selectedDeviceId")
GROUP BY p_deviceId

p_deviceId,avgTemp,stdTemp
3,24.5143,6.39


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>